# INST326 — Week 9 Exercises: Abstract Classes & Interfaces (Library Management)

**Focus (Week 9 only):** Abstract Base Classes (ABCs) with `abc.ABC` and `@abstractmethod`, abstract properties, virtual subclass registration, and light “interface-like” design via ABCs and (optional) `typing.Protocol` **without** advanced generics.

**Out of scope (Week 10+):** multiple inheritance/mixins, advanced design patterns, dependency injection, metaclasses beyond `ABCMeta`, decorators beyond basics, complex type-system features (ParamSpec, TypeVar variance), context managers beyond prior weeks.


### Starter Scaffold (Week-9-safe)

Below is a minimal domain model from prior weeks, slightly adapted for Week 9. We keep inheritance simple and introduce **abstract classes** to define common contracts.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Protocol
from abc import ABC, abstractmethod

# --- Exceptions (kept simple) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...
class NonBorrowableError(LibraryError): ...

# --- Abstract base for library items ---
class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    # A partial template method that depends on an abstract method.
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    # default stock policy common to most items
    def can_checkout(self) -> bool:
        return self.copies > 0

# --- Concrete items (will be extended in exercises) ---
class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    @property
    def is_digital(self) -> bool:
        return False

class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    def can_checkout(self) -> bool:
        return self.copies >= 0

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True

# --- Optional Protocol example (kept simple & non-generic) ---
class Downloadable(Protocol):
    def download_link(self) -> str: ...

# --- Core containers ---
@dataclass
class Member:
    member_id: str
    email: str
    def max_concurrent_loans(self) -> int:
        return 5

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False
    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._items: Dict[str, LibraryItem] = {}
    def add_item(self, item: LibraryItem) -> None:
        if item.isbn in self._items:
            raise DuplicateBookError(f"ISBN already exists: {item.isbn}")
        if item.copies < 0:
            raise ValueError("copies must be non-negative")
        self._items[item.isbn] = item
    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        return self._items.get(isbn)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []
    def active_loans_for(self, member: Member) -> List[Loan]:
        return [L for L in self.loans if (L.member_id == member.member_id and not L.returned)]
    def checkout(self, member: Member, item: LibraryItem) -> Loan:
        if len(self.active_loans_for(member)) >= member.max_concurrent_loans():
            raise LibraryError("concurrent loan limit reached")
        if not item.can_checkout():
            raise NonBorrowableError("cannot checkout under current stock policy")
        item.copies -= 1
        loan = Loan(isbn=item.isbn, member_id=member.member_id, due_date=item.due_date_from_today())
        self.loans.append(loan)
        return loan
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            it = self.catalog.get_item(loan.isbn)
            if it:
                it.copies += 1
            loan.mark_returned()


## 1) Make `LibraryItem` truly abstract

Prove that `LibraryItem` cannot be instantiated directly. Write a quick try/except demonstrating that creating `LibraryItem('x','y')` raises a `TypeError` because of abstract methods.

In [5]:
# Your code here
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass

    

# Demonstrate TypeError on instantiation of abstract class
try:
    item = LibraryItem('x', 'y', 'z')
except TypeError as e:
    print(f"Caught error {e}")

Caught error Can't instantiate abstract class LibraryItem without an implementation for abstract methods 'describe', 'is_digital', 'loan_period_days', 'media_type'


## 2) Abstract property practice

Add an **abstract property** `media_type` to `LibraryItem` and implement it in all concrete subclasses with strings like `'print'`, `'ebook'`, `'audiobook'`.

In [7]:
# Your code here
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    def media_type(self) -> str:
        return "ebook"
    

    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    



    

# Add @property @abstractmethod def media_type(self) -> str: ...

## 3) Abstract classmethod

Add an abstract `@classmethod def kind(cls) -> str` to `LibraryItem` returning a short identifier (e.g., `'book'`). Implement it for the concrete subclasses.

In [ ]:
# Your code here
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    

    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
    
    
# classmethod kind() -> str on LibraryItem and overrides

## 4) Template method using abstract hook

Create a template method `receipt_line(self) -> str` on `LibraryItem` that uses `self.describe()` and `self.loan_period_days()`. Show that each subclass inherits the same template but outputs different text due to overrides.

In [ ]:
# Your code here
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    

    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
# Implement receipt_line in LibraryItem using abstract hooks

## 5) Virtual subclass registration

Create a new class `PDFPamphlet` **without** inheriting from `LibraryItem`, but `register` it as a virtual subclass using `LibraryItem.register(PDFPamphlet)`. Implement the required interface manually. Show that `isinstance(pdf, LibraryItem)` returns `True` after registration.

In [11]:
# Your code here
class PDFPamphlet:
    def __init__(self, isbn: str, title: str, pages: int) -> None:
        self.isbn = isbn
        self.title = title
        self.pages = pages
        self.copies = 1

    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        return 7

    def describe(self) -> str:
        """Human-readable description of the item."""
        return f"PDFPamphlet<{self.isbn}>: {self.title} ({self.pages} pages)"

    @property
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        return True

    @property
    def media_type(self) -> str:
        return "pdf"


    @classmethod
    def kind(cls) -> str:
        return "pamphlet"
    

LibraryItem.register(PDFPamphlet)
pdf = PDFPamphlet("1111", "Sample Pamphlet", 10)
print(isinstance(pdf, LibraryItem))  

    
# Define PDFPamphlet, register as virtual subclass, demonstrate isinstance

True


## 6) Abstract property for availability

Add an abstract property `borrowable: bool` to `LibraryItem`. For `PrintedBook` and `AudioBook`, return `True`. For `EBook`, return `True` if `copies >= 0`. Demonstrate a check before `LoanDesk.checkout` that raises `NonBorrowableError` if `borrowable` is False.

In [13]:
# Your code here
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class NonBorrowableItemError(Exception):
    pass


class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"
    
    @property
    @abstractmethod
    def borrowable(self) -> bool:
        pass




class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"
    
    @property
    def borrowable(self) -> bool:
        return True


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    

    
class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
    @property
    def borrowable(self) -> bool:
        return True
    
class LoanDesk:
    @staticmethod
    def checkout(item: LibraryItem) -> None:
        if not item.borrowable:
            raise NonBorrowableItemError("This item cannot be borrowed.")
        print(f"Checked out: {item.describe()}")
    
# Add property and demonstrate guarding behavior

## 7) EBook implements Downloadable Protocol

Implement `download_link(self) -> str` on `EBook` to satisfy `Downloadable`. Write a function `offer_download(x)` that accepts a `Downloadable` and returns its link. Show duck-typed use with an `EBook` instance.

In [16]:
# Your code here
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Protocol
from abc import ABC, abstractmethod

class NonBorrowableItemError(Exception):
    pass

class Downloadable(Protocol):
    def download_link(self) -> str: 
        ...


class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"
    
    @property
    @abstractmethod
    def borrowable(self) -> bool:
        pass


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    
    def download_link(self) -> str:
        return f"https://example.com/download/{self.isbn}"
    
def offer_download(x: Downloadable) -> str:
    return x.download_link()
    
ebook = EBook("1234", "ebook", file_size_mb=5.8)
print(offer_download(ebook))
# EBook.download_link and offer_download(downloadable)

https://example.com/download/1234


## 8) Protocol vs ABC (short reflection)

In a short markdown cell, explain the difference between using an ABC and a Protocol for “interfaces” in Python, and when you might pick one over the other in this project.

In [ ]:
# (Write your reflection in this markdown cell.)
#ABC makes a class follow a certain structure by inherting from it. It's good to use when you want all subclasses
#to have same methods

#Protocol allows class to be used the same as having a certain behavior. This is if the methods are correct (doesn't need to inherit)
                                                                                                    
#For this project ABC is better for having a structure while Protocol is for behavior checking (more flexibility)

## 9) `LoanDesk` typed for abstraction

Refactor type hints in `LoanDesk` to accept `LibraryItem` rather than concrete classes everywhere. Explain (markdown) why depending on the abstract type improves flexibility.

In [17]:
# Your code here
# (Minor changes may already reflect this in the scaffold.)

class LoanDesk:
    @staticmethod
    def checkout(item: LibraryItem) -> None:
        if not item.borrowable:
            raise NonBorrowableItemError("This item cannot be borrowed.")
        print(f"Checked out: {item.describe()}")

## 10) Abstract fee policy

Add an abstract method `daily_late_fee(self) -> float` to `LibraryItem`. Implement fees:
- PrintedBook: 0.25
- EBook: 0.10
- AudioBook: 0.15
Add a concrete `late_fee(self, days_late: int) -> float` in `LibraryItem` that multiplies days by `daily_late_fee()`.

In [19]:
# Your code here
from datetime import datetime, timedelta
from abc import ABC, abstractmethod

class NonBorrowableItemError(Exception):
    pass

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"
    
    @property
    @abstractmethod
    def borrowable(self) -> bool:
        pass
    
    @abstractmethod
    def daily_late_fee(self) -> float:
        pass

    def late_fee(self, days_late: int) -> float:
        return days_late * self.daily_late_fee()
    

class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"
    
    @property
    def borrowable(self) -> bool:
        return True
    
    def daily_late_fee(self) -> float:
        return 0.25


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    def daily_late_fee(self) -> float:
        return 0.10
    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
    @property
    def borrowable(self) -> bool:
        return True
    def daily_late_fee(self) -> float:
        return 0.15
# Add abstract daily_late_fee and concrete late_fee to LibraryItem and overrides

## 11) ABC for Member roles

Create an abstract base `MemberRole(ABC)` with `max_concurrent_loans(self) -> int`. Implement `StudentRole` (5) and `StaffRole` (10). Modify `Member` to hold a `role: MemberRole` and delegate `max_concurrent_loans()` to it. Keep the implementation single-inheritance (no mixins).

In [20]:
# Your code here
from abc import ABC, abstractmethod

class MemberRole(ABC):
    @abstractmethod
    def max_concurrent_loans(self) -> int:
        pass

class StudentRole(MemberRole):
    def max_concurrent_loans(self) -> int:
        return 5
    
class StaffRole(MemberRole):
    def max_concurrent_loans(self) -> int:
        return 10
    
class Member:
    def __init__(self, name: str, role: MemberRole) -> None:
        self.name = name
        self.role = role

    def max_concurrent_loans(self) -> int:
        return self.role.max_concurrent_loans()
    

# Define roles and update Member

student = Member("Alex", StudentRole())
staff = Member("Hailey", StaffRole())

print(student.max_concurrent_loans())  
print(staff.max_concurrent_loans())    

5
10


## 12) Prevent partial implementations

Create a subclass `BrokenItem(LibraryItem)` that **forgets** to implement one abstract member. Show that instantiating it raises `TypeError`. Then fix it by implementing the missing member.

In [22]:
# Your code here
from abc import ABC, abstractmethod

class BrokenItem(LibraryItem):
    def loan_period_days(self) -> int:
        return 4
    
    def describe(self) -> str:
        return f"BrokenItem missing items"
    
    @property
    def is_digital(self) -> bool:
        return False
    

try:
    item = BrokenItem("4822", "Broken Item")
except TypeError as e:
    print("TypeError:", e)



class FixedItem(LibraryItem):
    def loan_period_days(self) -> int:
        return 4
    
    def describe(self) -> str:
        return f"FixedItem complete"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return "fixed"
    
    @classmethod
    def kind(cls) -> str:
        return "fixed"
    
    @property
    def borrowable(self) -> bool:
        return True
    
    def daily_late_fee(self) -> float:
        return 0.05
    
fixed = FixedItem("4822", "Fixed Item")
print(fixed.describe())
    

# Show failing instantiation then the fix

TypeError: Can't instantiate abstract class BrokenItem without an implementation for abstract methods 'borrowable', 'daily_late_fee', 'kind', 'media_type'
FixedItem complete


## 13) Abstract validation hook

Add an abstract hook `validate_on_add(self) -> None` to `LibraryItem` and override it in each subclass to enforce a simple constraint (e.g., `copies >= 0`). Modify `Catalog.add_item` to call `item.validate_on_add()` before insertion.

In [23]:
# Your code here
from abc import ABC, abstractmethod

class NonBorrowableItemError(Exception):
    pass

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"
    
    @property
    @abstractmethod
    def borrowable(self) -> bool:
        pass
    
    @abstractmethod
    def daily_late_fee(self) -> float:
        pass

    def late_fee(self, days_late: int) -> float:
        return days_late * self.daily_late_fee()
    
    @abstractmethod
    def validate_on_add(self) -> None:
        pass
    

class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"
    
    @property
    def borrowable(self) -> bool:
        return True
    
    def daily_late_fee(self) -> float:
        return 0.25
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    def daily_late_fee(self) -> float:
        return 0.10
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")
    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
    @property
    def borrowable(self) -> bool:
        return True
    def daily_late_fee(self) -> float:
        return 0.15
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")
        

class Catalog:
    def __init__(self):
        self.items = []

    def add_item(self, item: LibraryItem) -> None:
        item.validate_on_add()
        self.items.append(item)
        print(f"Added item: {item.describe()}")
# Add validate_on_add to classes and call from Catalog.add_item

## 14) Minimal adapter via ABC registration

Suppose you receive third-party objects with attributes `code`, `name`, `stock` that you want to treat as `LibraryItem`. Write a light Adapter class that **implements** the `LibraryItem` API and delegates to the 3rd-party object, then register it (or the 3rd-party class) appropriately. Show it working with `LoanDesk.checkout`.

In [24]:
# Your code here
class ThirdParty:
    def __init__(self, code: str, name: str, available_copies: int) -> None:
        self.code = code
        self.name = name
        self.available_copies = available_copies

class ThirdPartyAdapter(LibraryItem):
    def __init__(self, third_party: ThirdParty) -> None:
        super().__init__(third_party.code, third_party.name, third_party.available_copies)
        self.third_party = third_party

    def loan_period_days(self) -> int:
        return 10

    def describe(self) -> str:
        return f"ThirdPartyItem<{self.isbn}>: {self.title} (copies={self.copies})"

    @property
    def is_digital(self) -> bool:
        return False

    @property
    def media_type(self) -> str:
        return "thirdparty"

    @classmethod
    def kind(cls) -> str:
        return "thirdparty"

    @property
    def borrowable(self) -> bool:
        return self.copies > 0

    def daily_late_fee(self) -> float:
        return 0.30

    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")
# Implement Adapter that conforms to LibraryItem contract

## 15) Unit test: abstract contract

Using `unittest`, write tests that assert:
- `LibraryItem` instantiation fails
- All concrete classes implement `loan_period_days` and `describe`
- `late_fee` uses the subclass-specific `daily_late_fee` values

In [44]:
# Your code here
import unittest

class TestWeek9ABCs(unittest.TestCase):
    def test_abstract_instantiation(self):
        with self.assertRaises(TypeError):
            item = LibraryItem("1234", "Abstract")
    def test_concretes_implement_contract(self):
        items = [
            PrintedBook("2345", "Printed Book"),
            EBook("4983", "EBook", file_size_mb=3.5),
            AudioBook("8943", "Audio Book", duration_min=120),
        ]
        for item in items:
            self.assertIsInstance(item.loan_period_days(), int)
            self.assertIsInstance(item.describe(), str)
    
    def test_late_fee_polymorphism(self):
        book = PrintedBook("2345", "Printed Book", copies=1)
        ebook = EBook("4983", "EBook", copies=1, file_size_mb=3.5)
        audiobook = AudioBook("8943", "Audio Book", copies=1, duration_min=120)
        self.assertEqual(book.late_fee(3), 0.75)
        self.assertEqual(ebook.late_fee(3), 0.30)
        self.assertEqual(audiobook.late_fee(3), 0.45)

# # To run tests in notebook:
# # unittest.main(argv=['-v'], exit=False)

## 16) Swap implementation behind the ABC

Write a function `checkout_any(desk: LoanDesk, member: Member, item: LibraryItem)` that works for **any** `LibraryItem` or registered virtual subclass. Demonstrate with a `PDFPamphlet` instance (from Ex. 5) and a normal `PrintedBook`.

In [26]:
# Your code here
class PDFPamphlet:
    def __init__(self, code: str, name: str, stock: int, pages: int) -> None:
        self.code = code
        self.name = name
        self.stock = stock
        self.pages = pages

    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        return 7

    def describe(self) -> str:
        """Human-readable description of the item."""
        return f"PDFPamphlet<{self.isbn}>: {self.title} ({self.pages} pages)"

    @property
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        return True

    @property
    def media_type(self) -> str:
        return "pdf"


    @classmethod
    def kind(cls) -> str:
        return "pdf"
    
    @property
    def borrowable(self) -> bool:
        return self.stock > 0
    
    def daily_late_fee(self) -> float:
        return 0.08
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")
        
    @property
    def isbn(self):
        return self.code
    
    @property
    def title(self):
        return self.name
    
    @property
    def copies(self):
        return self.stock
    

LibraryItem.register(PDFPamphlet)

class LoanDesk:
    @staticmethod
    def checkout(item: LibraryItem) -> None:
        if not item.borrowable:
            raise NonBorrowableItemError("This item cannot be borrowed.")
        print(f"Checked out: {item.describe()}")

class Member:
    def __init__(self, member_id: str) -> None:
        self.member_id = member_id

def checkout_any(desk: LoanDesk, member: Member, item: LibraryItem):
    desk.checkout(item)

desk = LoanDesk()
member = Member("Alex")

real_item = PrintedBook("5555", "Real Book", copies=2)
virtual_item = PDFPamphlet("6666", "Virtual Pamphlet", stock=15, pages =20)



# Demo with virtual subclass and a real subclass

## 17) Abstract property + computed template

Add a template method `full_label()` to `LibraryItem` that returns `f"[{self.media_type}] {self.title} — {self.isbn}"`. Confirm each subclass inherits it and shows the right `media_type` value.

In [31]:
# Your code here
from abc import ABC, abstractmethod

class NonBorrowableItemError(Exception):
    pass

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        pass

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""
        pass

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        pass

    @property
    @abstractmethod
    def media_type(self) -> str:
        pass


    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        pass

    def receipt_line(self) -> str:
        return f"{self.describe()} - Loan for {self.loan_period_days()} days"
    
    def full_label(self) -> str:
        return f"[{self.media_type}] {self.title} - {self.isbn}"
    
    @property
    @abstractmethod
    def borrowable(self) -> bool:
        pass
    
    @abstractmethod
    def daily_late_fee(self) -> float:
        pass

    def late_fee(self, days_late: int) -> float:
        return days_late * self.daily_late_fee()
    

class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    

    @property
    def media_type(self) -> str:
        return "print"
    

    @classmethod
    def kind(cls) -> str:
        return "book"
    
    @property
    def borrowable(self) -> bool:
        return True
    
    def daily_late_fee(self) -> float:
        return 0.25


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    def daily_late_fee(self) -> float:
        return 0.10
    

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audiobook"
    @property
    def borrowable(self) -> bool:
        return True
    def daily_late_fee(self) -> float:
        return 0.15
    

class PDFPamphlet(LibraryItem):
    def __init__(self, code: str, name: str, stock: int, pages: int) -> None:
        self.code = code
        self.name = name
        self.stock = stock
        self.pages = pages

    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""
        return 7

    def describe(self) -> str:
        """Human-readable description of the item."""
        return f"PDFPamphlet<{self.isbn}>: {self.title} ({self.pages} pages)"

    @property
    def is_digital(self) -> bool:
        """Whether the item is digital."""
        return True

    @property
    def media_type(self) -> str:
        return "pdf"


    @classmethod
    def kind(cls) -> str:
        return "pdf"
    
    @property
    def borrowable(self) -> bool:
        return self.stock > 0
    
    def daily_late_fee(self) -> float:
        return 0.08
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("Can't be negative")
        
    @property
    def isbn(self):
        return self.code
    
    @property
    def title(self):
        return self.name
    
    @property
    def copies(self):
        return self.stock
    

LibraryItem.register(PDFPamphlet)


print(PrintedBook("1233","Book", 2).full_label())
print(EBook("2344","EBook", 1, 5.0).full_label())
print(AudioBook("3455","Audio", 3, 60).full_label())
print(PDFPamphlet("4566","Pamphlet", 4, 15).full_label())
# Implement full_label in LibraryItem and demonstrate across subclasses

[print] Book - 1233
[ebook] EBook - 2344
[audiobook] Audio - 3455
[pdf] Pamphlet - 4566


## 18) issubclass / isinstance with ABCs

Show examples of `issubclass(PrintedBook, LibraryItem)` and `isinstance(EBook(...), LibraryItem)`. After registering a virtual subclass, show `issubclass(PDFPamphlet, LibraryItem)` is `True` as well.

In [38]:
# Your code here
# Demonstrate issubclass/isinstance facts
# Demonstrate issubclass/isinstance facts

print(issubclass(PrintedBook, LibraryItem))
print(isinstance(EBook("3821", "ebook", 1, 5.0), LibraryItem)) 

print(issubclass(EBook, LibraryItem)) 

print(issubclass(PDFPamphlet, LibraryItem)) 
print(isinstance(PDFPamphlet("1111", "Pamphlet", 10, 12), LibraryItem))


True
True
True
True
True


## 19) Inventory report via abstraction

Write `summarize_items(items: list[LibraryItem]) -> list[str]` that uses only abstract methods/properties (`describe`, `media_type`, etc.). Demonstrate polymorphism by passing a mixed list of items.

In [40]:
# Your code here
def summarize_items(items: list[LibraryItem]) -> list[str]:
    return [f"{item.media_type}: {item.describe()}" for item in items]

items = [
    PrintedBook("1111", "Book", 3),
    EBook("2222", "EBook", 2, 4.5),
    AudioBook("3333", "AudioBook", 1, 90),
    PDFPamphlet("4444", "Pamphlet", 5, 20)
]

summaries = summarize_items(items)
for summary in summaries:
    print(summary)

print: PrintedBook<1111>: Book (copies=3)
ebook: EBook<2222>: EBook (4.5 MB)
audiobook: AudioBook<3333>: AudioBook (90 min)
pdf: PDFPamphlet<4444>: Pamphlet (20 pages)


## 20) End-to-end scenario under ABC contract

Create a demo that:
- Builds a `Catalog` and `LoanDesk`
- Adds one of each concrete item and one registered virtual subclass instance
- Checks each out to a `Member`
- Prints a small receipt using `receipt_line()` and the computed due dates
Use only `LibraryItem`-level APIs at call sites (no `isinstance` branches).

In [43]:
# Your code here
# End-to-end demo using the abstract contract only

from datetime import date, timedelta

class Catalog:
    def __init__(self) -> None:
        self.items = {}

    def add_item(self, item) -> None:
        if hasattr(item, 'validate_on_add'):
            item.validate_on_add()
        self.items[item.isbn] = item

    def get(self, isbn: str):
        return self.items.get(isbn)

class Member:
    def __init__(self, name: str) -> None:
        self.name = name
        self.loans = []

    def checkout(self, item, loan_date: date):
        if not item.borrowable:
            raise NonBorrowableItemError(f"Item '{item.title}' is not borrowable.")
        self.loans.append((item, loan_date))

class LoanDesk:
    def __init__(self, catalog: Catalog) -> None:
        self.catalog = catalog

    def checkout(self, member: Member, isbn: str, loan_date: date):
        item = self.catalog.get(isbn)
        if item is None:
            raise ValueError("Item not found.")
        member.checkout(item, loan_date)

def run_end_to_end_demo():
    today = date.today()

    catalog = Catalog()
    desk = LoanDesk(catalog)

    items = [
        PrintedBook("1111", "Printed Book", 3),
        EBook("2222", "Digital EBook", 2, 4.5),
        AudioBook("3333", "Narrated Audio", 1, 90),
        PDFPamphlet("4444", "Short PDF", 5, 20)
    ]

    for item in items:
        catalog.add_item(item)

    member = Member("Alice")

    for item in items:
        desk.checkout(member, item.isbn, today)

    print(f"Receipt for {member.name}:")
    for item, loan_date in member.loans:
        due_date = loan_date + timedelta(days=item.loan_period_days())
        print(f"- {item.receipt_line()} | Due: {due_date}")

run_end_to_end_demo()
    



Receipt for Alice:
- PrintedBook<1111>: Printed Book (copies=3) - Loan for 21 days | Due: 2025-12-07
- EBook<2222>: Digital EBook (4.5 MB) - Loan for 14 days | Due: 2025-11-30
- AudioBook<3333>: Narrated Audio (90 min) - Loan for 14 days | Due: 2025-11-30
- PDFPamphlet<4444>: Short PDF (20 pages) - Loan for 7 days | Due: 2025-11-23


## Python skills you'll need (Weeks 1–9)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts (basic use), simple comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write, simple JSON usage
- **Classes & objects (Weeks 4–8):** classes, `__init__`, instance methods, overriding, `super()`
- **Encapsulation basics:** simple validation; naming conventions for "private" attributes
- **Error handling & testing (Week 7):** `try/except`, custom exceptions, basic `unittest`
- **Week 8 OOP:** single inheritance & polymorphism (no ABCs)
- **Week 9 focus:** **Abstract Base Classes (ABC)** with `abc.ABC` and `@abstractmethod`, abstract properties, class/instance abstract methods, virtual subclass **registration**, and light `typing.Protocol` usage (non-generic)
- **Standard library familiarity:** `abc`, `datetime`, built-in exceptions
